# Sentiment Classification
이 곳에 작성된 소스코드는 https://github.com/czangyeob/ACORN_DLKERAS/blob/master/Chapter06/umich_sentiment_lstm.py 를 기반으로 작성되었습니다

In [1]:
# -*- coding: utf-8 -*-
from keras.layers.core import Activation, Dense, Dropout, SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os

DATA_DIR = "./data"

EMBEDDING_SIZE = 128
HIDDEN_LAYER_SIZE = 64
BATCH_SIZE = 32
NUM_EPOCHS = 10

# 학습 데이터를 읽고 사전을 만든다.
maxlen = 0
word_freqs = collections.Counter()
num_recs = 0
ftrain = open(os.path.join(DATA_DIR, "umich-sentiment-train.txt"), 'r')
for line in ftrain:
    label, sentence = line.strip().split("\t")
    words = nltk.word_tokenize(sentence.lower())
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        word_freqs[word] += 1
    num_recs += 1
ftrain.close()

vocab_size = len(word_freqs) + 2
word2index = {x[0]: i+2 for i, x in 
                enumerate(word_freqs.most_common(vocab_size))}
word2index["PAD"] = 0
word2index["UNK"] = 1
index2word = {v:k for k, v in word2index.items()}

# 문장을 시퀀스로 변환
X = np.empty((num_recs, ), dtype=list)
y = np.zeros((num_recs, ))
i = 0
ftrain = open(os.path.join(DATA_DIR, "umich-sentiment-train.txt"), 'r')
for line in ftrain:
    label, sentence = line.strip().split("\t")
    words = nltk.word_tokenize(sentence.lower())
    seqs = []
    for word in words:
        #if word2index.has_key(word):
        if word in word2index:
            seqs.append(word2index[word])
        else:
            seqs.append(word2index["UNK"])
    X[i] = seqs
    y[i] = int(label)
    i += 1
ftrain.close()

X = sequence.pad_sequences(X, maxlen=maxlen)
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, 
                                                random_state=42)

# 모델 작성
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(HIDDEN_LAYER_SIZE, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", 
              metrics=["accuracy"])

history = model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, 
                    epochs=NUM_EPOCHS,
                    validation_data=(Xtest, ytest))

# 손실과 정확도 시각화
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["acc"], color="g", label="Train")
plt.plot(history.history["val_acc"], color="b", label="Validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="Train")
plt.plot(history.history["val_loss"], color="b", label="Validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()

# 평가
score, acc = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
print("Test score: %.3f, accuracy: %.3f" % (score, acc))

for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,42)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0].tolist() if x != 0])
    print("%.0f\t%d\t%s" % (ypred, ylabel, sent))

Using TensorFlow backend.


Train on 5668 samples, validate on 1418 samples
Epoch 1/10
5668/5668 [==============================] - 20s 4ms/step - loss: 0.2429 - acc: 0.8948 - val_loss: 0.0724 - val_acc: 0.9711
Epoch 2/10
5668/5668 [==============================] - 19s 3ms/step - loss: 0.0304 - acc: 0.9898 - val_loss: 0.0466 - val_acc: 0.9845
Epoch 3/10
5668/5668 [==============================] - 19s 3ms/step - loss: 0.0112 - acc: 0.9968 - val_loss: 0.0462 - val_acc: 0.9838
Epoch 4/10
5668/5668 [==============================] - 19s 3ms/step - loss: 0.0036 - acc: 0.9995 - val_loss: 0.0463 - val_acc: 0.9873
Epoch 5/10
5668/5668 [==============================] - 19s 3ms/step - loss: 0.0035 - acc: 0.9989 - val_loss: 0.0572 - val_acc: 0.9831
Epoch 6/10
5668/5668 [==============================] - 19s 3ms/step - loss: 0.0021 - acc: 0.9995 - val_loss: 0.0494 - val_acc: 0.9866
Epoch 7/10
5668/5668 [==============================] - 19s 3ms/step - loss: 0.0024 - acc: 0.9995 - val_loss: 0.0597 - val_acc: 0.9859
Epoch 8

<Figure size 640x480 with 2 Axes>

1418/1418 [==============================] - 0s 210us/step
Test score: 0.062, accuracy: 0.989
0	0	everyone knows brokeback mountain is going to win all because of the stupid gay cowboys .
1	1	love luv lubb the da vinci code !
1	1	love luv lubb the da vinci code !
0	0	i hate harry potter .
0	0	so brokeback mountain was really depressing .
